In [2]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/m2cw

Mounted at /content/drive
/content/drive/MyDrive/m2cw


In [3]:
import h5py
import numpy as np
import torch
from transformers import AutoTokenizer
import wandb
import matplotlib.pyplot as plt
from preprocessor import load_and_preprocess
from lora_skeleton import apply_lora, train_lora,load_data
from qwen import load_qwen
from evaluation import evaluation

if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")

wandb.login()



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: liuyihao649 (liuyihao649-university-of-cambridge) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [4]:
# Load Qwen2.5 0.5B Model and Tokenizer ---
model_name = "Qwen/Qwen2.5-0.5B"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

file_path = "lotka_volterra_data.h5"

# Use the function to load and preprocess the data
train_texts, val_texts, test_texts = load_and_preprocess(
    file_path,
    decimal_places=2,
    max_target_value=9.99
)

# Demonstrate tokenization using Qwen2.5
model_name = "Qwen/Qwen2.5-0.5B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

tokenized_train=[]
tokenized_val=[]
tokenized_test=[]
for i in range(len(train_texts)):
    tokenized_train.append(tokenizer(train_texts[i], return_tensors="pt", add_special_tokens=False)["input_ids"][0])
for i in range(len(val_texts)):
    tokenized_val.append(tokenizer(val_texts[i], return_tensors="pt", add_special_tokens=False)["input_ids"][0])
for i in range(len(test_texts)):
    tokenized_test.append(tokenizer(test_texts[i], return_tensors="pt", add_special_tokens=False)["input_ids"][0])



tokenizer_config.json:   0%|          | 0.00/7.23k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

In [5]:
train_ids, val_ids, test_ids = load_data(tokenizer)

### 3(b): Grid search

In [6]:
for rank in [2, 4, 8]:
    for lr in [1e-5, 5e-5, 1e-4]:
        # init each experiments
        wandb.init(
            project="m2-lora-grid-search",
            name=f"rank{rank}_lr{lr}_train",
            group="train_loss_curve",
            reinit=True
        )

        # load the original model and apply lora to it
        base_model,_ = load_qwen()
        apply_lora(base_model, r=rank)



        # LoRA train
        _ = train_lora(
            base_model,
            train_ids,
            learning_rate=lr,
            batch_size=4,
            max_steps=1000,

        )

        # Save the weights(only LoRA part, no need to save the whole model)
        lora_state_dict = {
        k: v for k, v in base_model.state_dict().items() if "lora_" in k
        }

        save_path = f"/content/drive/MyDrive/m2cw/lora_rank{rank}_lr{lr}.pt"
        torch.save(lora_state_dict, save_path)

        # Evaluation
        wandb.init(project="m2-lora-grid-search",
            name=f"rank{rank}_lr{lr}_evaluation",
            reinit=True)
        val_loss, val_mse = evaluation(base_model, tokenizer, tokenized_val)


        print(f"rank{rank},lr{lr},evalaution results:")
        print(f"Average Cross-Entropy Loss: {val_loss:.4f}")
        print(f"Average MSE (Forecast):  {val_mse:.4f}")



        wandb.finish()

Training:   5%|▌         | 50/1000 [00:15<04:37,  3.42it/s, loss=1.04]

Step 50: loss = 1.0398


Training:  10%|█         | 100/1000 [00:30<04:22,  3.43it/s, loss=0.929]

Step 100: loss = 0.9291


Training:  15%|█▌        | 150/1000 [00:44<04:08,  3.42it/s, loss=0.719]

...

Step 1000: loss = 0.3052


loss,██▇▅▅▃▄▃▃▄▂▃▃▁▂▃▃▃▂▂▂▁▂▂▂▁▂▂▄▂▁▃▁▁▂▃▂▂▂▁
step,▁▁▁▁▂▂▂▃▃▃▄▄▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇▇██
loss,0.30516
step,999


wandb: WARNING Tried to log to step 1 that is less than the current step 2. Steps must be monotonically increasing, so this data will be ignored. See https://wandb.me/define-metric to log data out of order.
wandb: WARNING Tried to log to step 2 that is less than the current step 4. Steps must be monotonically increasing, so this data will be ignored. See https://wandb.me/define-metric to log data out of order.
wandb: WARNING Tried to log to step 3 that is less than the current step 6. Steps must be monotonically increasing, so this data will be ignored. See https://wandb.me/define-metric to log data out of order.
wandb: WARNING Tried to log to step 4 that is less than the current step 8. Steps must be monotonically increasing, so this data will be ignored. See https://wandb.me/define-metric to log data out of order.
wandb: WARNING Tried to log to step 5 that is less than the current step 10. Steps must be monotonically increasing, so this data will be ignored. See https://wandb.me/defi

rank8,lr0.0001,evalaution results:
Average Cross-Entropy Loss: 1.8566
Average MSE (Forecast):  0.0148


avg_loss,▁
avg_mse,▁
avg_loss,1.85659
avg_mse,0.0148


In [9]:
max_context_length=[128,512,768]

for length in max_context_length:
    train_ids,_,_=load_data(tokenizer,max_ctx_length=length)
    wandb.init(
        project="m2-lora-different-context-len",
        name=f"context-length:{length}_train",
        group="train_loss_curve",
        reinit=True
    )
    base_model,_ = load_qwen()
    apply_lora(base_model)
    _ = train_lora(
        base_model,
        train_ids,
        learning_rate=1e-5,
        batch_size=4,
        max_steps=1000,
    )



    wandb.init(
        project="m2-lora-different-context-len",
        name=f"context-length:{length}_evaluation",
        reinit=True
    )
    val_loss, val_mse = evaluation(base_model, tokenizer, tokenized_val)


    print(f"rank{rank},lr{lr},evalaution results:")
    print(f"Average Cross-Entropy Loss: {val_loss:.4f}")
    print(f"Average MSE (Forecast):  {val_mse:.4f}")

    wandb.finish()


Training:   5%|▌         | 51/1000 [00:05<01:49,  8.67it/s, loss=0.886]

Step 50: loss = 0.9314


Training:  10%|█         | 101/1000 [00:11<01:40,  8.97it/s, loss=0.84] 

Step 100: loss = 0.8558


Training:  15%|█▌        | 151/1000 [00:17<01:34,  8.98it/s, loss=0.907]

...

Step 1000: loss = 0.4363


loss,▄▇█▇▆▇▆▇▇█▇▅▅▅▅▄▄▄▆▅▅▃▃▃▆▄▄▃▆▄▃▄▃▄▂▃▁▃▄▃
step,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇███
loss,0.43626
step,999


wandb: WARNING Tried to log to step 1 that is less than the current step 2. Steps must be monotonically increasing, so this data will be ignored. See https://wandb.me/define-metric to log data out of order.
wandb: WARNING Tried to log to step 2 that is less than the current step 4. Steps must be monotonically increasing, so this data will be ignored. See https://wandb.me/define-metric to log data out of order.
wandb: WARNING Tried to log to step 3 that is less than the current step 6. Steps must be monotonically increasing, so this data will be ignored. See https://wandb.me/define-metric to log data out of order.
wandb: WARNING Tried to log to step 4 that is less than the current step 8. Steps must be monotonically increasing, so this data will be ignored. See https://wandb.me/define-metric to log data out of order.
wandb: WARNING Tried to log to step 5 that is less than the current step 10. Steps must be monotonically increasing, so this data will be ignored. See https://wandb.me/defi

rank8,lr0.0001,evalaution results:
Average Cross-Entropy Loss: 1.4701
Average MSE (Forecast):  0.0273


avg_loss,▁
avg_mse,▁
avg_loss,1.47013
avg_mse,0.02732


In [11]:
wandb.init(
    project="m2-lora-final-long-run-8-1e-4-768",
        name=f"final_train",
        reinit=True)
train_ids,_,_=load_data(tokenizer,max_ctx_length=768)

base_model,_ = load_qwen()

apply_lora(base_model,r=8)

_ = train_lora(
    base_model,
    train_ids,
    learning_rate=1e-4,
    batch_size=4,
    max_steps=3000
)



wandb.init(
    project="m2-lora-different-context-len",
    name=f"final_evaluation",
    reinit=True
)
val_loss, val_mse = evaluation(base_model, tokenizer, tokenized_val)


print(f"final evalaution results:")
print(f"Average Cross-Entropy Loss: {val_loss:.4f}")
print(f"Average MSE (Forecast):  {val_mse:.4f}")

wandb.finish()


Training:   2%|▏         | 50/3000 [00:21<20:41,  2.38it/s, loss=0.468]

Step 50: loss = 0.4675


Training:   3%|▎         | 100/3000 [00:42<20:18,  2.38it/s, loss=0.513]

Step 100: loss = 0.5129


Training:   5%|▌         | 150/3000 [01:03<19:57,  2.38it/s, loss=0.405]

...

Step 2200: loss = 0.3362


Training:  75%|███████▌  | 2250/3000 [15:46<05:14,  2.38it/s, loss=0.342]

Step 2250: loss = 0.3416


Training:  76%|███████▌  | 2278/3000 [15:58<05:03,  2.38it/s, loss=0.32] 

KeyboardInterrupt: 

In [12]:
wandb.init(
            project="m2-lora-grid-search",
            name=f"rank2_lr1e-4_train_2",
            reinit=True
        )

# load the original model and apply lora to it
base_model,_ = load_qwen()
apply_lora(base_model, r=2)



# LoRA train
_ = train_lora(
    base_model,
    train_ids,
    learning_rate=1e-4,
    batch_size=4,
    max_steps=1000,

)


# Evaluation
wandb.init(project="m2-lora-grid-search",
    name=f"rank2_lr1e-4_evaluation",
    reinit=True)
val_loss, val_mse = evaluation(base_model, tokenizer, tokenized_val)


print(f"rank{rank},lr{lr},evalaution results:")
print(f"Average Cross-Entropy Loss: {val_loss:.4f}")
print(f"Average MSE (Forecast):  {val_mse:.4f}")

loss,██▄▅▅█▆▃▆▆▃▆▄▄▄▅▅▃▆▃▅▃▃▄▃▂▄▄▅▅▃▄▂▃▄▅▃▁▂▄
step,▁▁▁▁▁▂▃▃▃▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇▇██
loss,0.32014
step,2277



Training:   5%|▌         | 50/1000 [00:21<06:44,  2.35it/s, loss=0.797]

Training:   5%|▌         | 50/1000 [00:21<06:44,  2.35it/s, loss=0.719]

Step 50: loss = 0.7192



Training:  10%|█         | 100/1000 [00:42<06:22,  2.35it/s, loss=0.509]

Training:  10%|█         | 100/1000 [00:42<06:22,  2.35it/s, loss=0.457]

Step 100: loss = 0.4574


...


Training: 100%|██████████| 1000/1000 [07:05<00:00,  2.35it/s, loss=0.415]

Training: 100%|██████████| 1000/1000 [07:05<00:00,  2.35it/s, loss=0.355]

Step 1000: loss = 0.3547


loss,█▇▅▆▄▆▄▄▄▄▄▄▄▃▃▄▄▃▁▃▄▃▅▄▃▄▃▃▂▃▃▂▃▄▂▃▃▂▃▂
step,▁▁▁▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,0.35471
step,999


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:650: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `20` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(
wandb: WARNING Tried to log to step 1 that is less than the current step 2. Steps must be monotonical

rank8,lr0.0001,evalaution results:
Average Cross-Entropy Loss: 1.5033
Average MSE (Forecast):  0.0103
